In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
print(mnist.train.num_examples) # Number of training data
print(mnist.test.num_examples) # Number of test data

/anaconda3/envs/tf-cpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
55000
10000


In [2]:
# Hyper-parameters
learningrate = 0.002
num_of_epochs=50
batchsize = 128
num_of_iters=55000/batchsize*num_of_epochs


noutput = 10

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, noutput])

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = tf.nn.relu(conv1)
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = tf.nn.relu(conv2)
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

init_var=tf.contrib.layers.variance_scaling_initializer()
init_xavier=tf.contrib.layers.xavier_initializer()

weights = {
    'wc1': tf.get_variable("wc1", shape=[5, 5, 1, 32],initializer=init_var),
    'wc2': tf.get_variable("wc2", shape=[5, 5, 32, 64],initializer=init_var),
    'wd1': tf.get_variable("wd1", shape=[7*7*64, 1024],initializer=init_var),
    'out': tf.get_variable("out", shape=[1024,noutput],initializer=init_var)
}

biases = {
    'bc1': tf.Variable(tf.zeros([32])),
    'bc2': tf.Variable(tf.zeros([64])),
    'bd1': tf.Variable(tf.zeros([1024])),
    'out': tf.Variable(tf.zeros([noutput]))
}

In [4]:
model = conv_net(X, weights, biases, dropout=0.5)


In [5]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels=Y))
#optimizer = tf.train.AdamOptimizer(learning_rate=learningrate)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer=tf.train.MomentumOptimizer(learning_rate=learningrate, momentum=0.99)

train_min = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


tf.summary.scalar("trainingLoss", loss)
merged_summary_op = tf.summary.merge_all()
filename = "./log/mnist_project_batchsize128_lr0p002" 
writer = tf.summary.FileWriter(filename, tf.get_default_graph())

init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    step=1
    while step <= num_of_iters:       
        batch_x, batch_y = mnist.train.next_batch(batchsize)  
        sess.run(train_min, feed_dict={X:batch_x, Y:batch_y})        
        losscalc, accuracycalc, merged_summary = sess.run([loss, accuracy,merged_summary_op], feed_dict={X:batch_x, Y:batch_y})
        if (step%10==1):
            print("Iteration: %d, Training Loss: %0.4f, Training Accuracy: %0.4f"%(step, losscalc, accuracycalc))
        writer.add_summary(merged_summary, step)
        step += 1
        
    test_x, test_y = mnist.test.next_batch(10000)
    losscalc, accuracycalc = sess.run([loss, accuracy], feed_dict={X:test_x, Y:test_y})
    print("Iterations: %d, Test Loss: %0.4f, Test Accuracy: %0.4f"%(step, losscalc, accuracycalc))

writer.close()               

Iteration: 1, Training Loss: 2.5624, Training Accuracy: 0.1094
Iteration: 11, Training Loss: 1.6846, Training Accuracy: 0.5000
Iteration: 21, Training Loss: 1.1829, Training Accuracy: 0.6484
Iteration: 31, Training Loss: 0.7664, Training Accuracy: 0.7578
Iteration: 41, Training Loss: 0.7071, Training Accuracy: 0.7969
Iteration: 51, Training Loss: 0.5944, Training Accuracy: 0.8047
Iteration: 61, Training Loss: 0.4489, Training Accuracy: 0.8984
Iteration: 71, Training Loss: 0.5697, Training Accuracy: 0.8516
Iteration: 81, Training Loss: 0.2370, Training Accuracy: 0.9375
Iteration: 91, Training Loss: 0.3530, Training Accuracy: 0.8828
Iteration: 101, Training Loss: 0.4487, Training Accuracy: 0.8750
Iteration: 111, Training Loss: 0.2382, Training Accuracy: 0.9141
Iteration: 121, Training Loss: 0.2812, Training Accuracy: 0.9062
Iteration: 131, Training Loss: 0.2764, Training Accuracy: 0.9375
Iteration: 141, Training Loss: 0.2275, Training Accuracy: 0.8984
Iteration: 151, Training Loss: 0.292

Iteration: 1261, Training Loss: 0.0166, Training Accuracy: 0.9922
Iteration: 1271, Training Loss: 0.0150, Training Accuracy: 0.9922
Iteration: 1281, Training Loss: 0.0140, Training Accuracy: 1.0000
Iteration: 1291, Training Loss: 0.0528, Training Accuracy: 0.9766
Iteration: 1301, Training Loss: 0.0549, Training Accuracy: 0.9844
Iteration: 1311, Training Loss: 0.0785, Training Accuracy: 0.9766
Iteration: 1321, Training Loss: 0.0355, Training Accuracy: 0.9922
Iteration: 1331, Training Loss: 0.0887, Training Accuracy: 0.9766
Iteration: 1341, Training Loss: 0.0094, Training Accuracy: 1.0000
Iteration: 1351, Training Loss: 0.0231, Training Accuracy: 0.9922
Iteration: 1361, Training Loss: 0.0089, Training Accuracy: 1.0000
Iteration: 1371, Training Loss: 0.1072, Training Accuracy: 0.9766
Iteration: 1381, Training Loss: 0.0057, Training Accuracy: 1.0000
Iteration: 1391, Training Loss: 0.0043, Training Accuracy: 1.0000
Iteration: 1401, Training Loss: 0.0361, Training Accuracy: 0.9922
Iteration:

Iteration: 2511, Training Loss: 0.0025, Training Accuracy: 1.0000
Iteration: 2521, Training Loss: 0.0345, Training Accuracy: 0.9766
Iteration: 2531, Training Loss: 0.0509, Training Accuracy: 0.9844
Iteration: 2541, Training Loss: 0.0483, Training Accuracy: 0.9922
Iteration: 2551, Training Loss: 0.0063, Training Accuracy: 1.0000
Iteration: 2561, Training Loss: 0.0052, Training Accuracy: 1.0000
Iteration: 2571, Training Loss: 0.0162, Training Accuracy: 0.9922
Iteration: 2581, Training Loss: 0.0389, Training Accuracy: 0.9844
Iteration: 2591, Training Loss: 0.0026, Training Accuracy: 1.0000
Iteration: 2601, Training Loss: 0.0031, Training Accuracy: 1.0000
Iteration: 2611, Training Loss: 0.0025, Training Accuracy: 1.0000
Iteration: 2621, Training Loss: 0.0080, Training Accuracy: 1.0000
Iteration: 2631, Training Loss: 0.0196, Training Accuracy: 0.9922
Iteration: 2641, Training Loss: 0.0062, Training Accuracy: 1.0000
Iteration: 2651, Training Loss: 0.0104, Training Accuracy: 1.0000
Iteration:

Iteration: 3761, Training Loss: 0.0256, Training Accuracy: 0.9922
Iteration: 3771, Training Loss: 0.0030, Training Accuracy: 1.0000
Iteration: 3781, Training Loss: 0.0044, Training Accuracy: 1.0000
Iteration: 3791, Training Loss: 0.0070, Training Accuracy: 1.0000
Iteration: 3801, Training Loss: 0.0073, Training Accuracy: 0.9922
Iteration: 3811, Training Loss: 0.0063, Training Accuracy: 1.0000
Iteration: 3821, Training Loss: 0.0271, Training Accuracy: 0.9922
Iteration: 3831, Training Loss: 0.0117, Training Accuracy: 1.0000
Iteration: 3841, Training Loss: 0.0011, Training Accuracy: 1.0000
Iteration: 3851, Training Loss: 0.0459, Training Accuracy: 0.9844
Iteration: 3861, Training Loss: 0.0030, Training Accuracy: 1.0000
Iteration: 3871, Training Loss: 0.0225, Training Accuracy: 0.9844
Iteration: 3881, Training Loss: 0.0082, Training Accuracy: 1.0000
Iteration: 3891, Training Loss: 0.0203, Training Accuracy: 0.9922
Iteration: 3901, Training Loss: 0.0148, Training Accuracy: 1.0000
Iteration:

Iteration: 5011, Training Loss: 0.0002, Training Accuracy: 1.0000
Iteration: 5021, Training Loss: 0.0030, Training Accuracy: 1.0000
Iteration: 5031, Training Loss: 0.0080, Training Accuracy: 1.0000
Iteration: 5041, Training Loss: 0.0201, Training Accuracy: 0.9922
Iteration: 5051, Training Loss: 0.0017, Training Accuracy: 1.0000
Iteration: 5061, Training Loss: 0.0015, Training Accuracy: 1.0000
Iteration: 5071, Training Loss: 0.0017, Training Accuracy: 1.0000
Iteration: 5081, Training Loss: 0.0152, Training Accuracy: 0.9922
Iteration: 5091, Training Loss: 0.0033, Training Accuracy: 1.0000
Iteration: 5101, Training Loss: 0.0129, Training Accuracy: 0.9922
Iteration: 5111, Training Loss: 0.0031, Training Accuracy: 1.0000
Iteration: 5121, Training Loss: 0.0072, Training Accuracy: 1.0000
Iteration: 5131, Training Loss: 0.0063, Training Accuracy: 1.0000
Iteration: 5141, Training Loss: 0.0156, Training Accuracy: 0.9922
Iteration: 5151, Training Loss: 0.0125, Training Accuracy: 0.9922
Iteration:

...

Iteration: 16161, Training Loss: 0.0007, Training Accuracy: 1.0000
Iteration: 16171, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 16181, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 16191, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 16201, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 16211, Training Loss: 0.0008, Training Accuracy: 1.0000
Iteration: 16221, Training Loss: 0.0005, Training Accuracy: 1.0000
Iteration: 16231, Training Loss: 0.0113, Training Accuracy: 0.9922
Iteration: 16241, Training Loss: 0.0007, Training Accuracy: 1.0000
Iteration: 16251, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 16261, Training Loss: 0.0019, Training Accuracy: 1.0000
Iteration: 16271, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 16281, Training Loss: 0.0002, Training Accuracy: 1.0000
Iteration: 16291, Training Loss: 0.0003, Training Accuracy: 1.0000
Iteration: 16301, Training Loss: 0.0002, Training Accuracy: 1.

Iteration: 17391, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 17401, Training Loss: 0.0012, Training Accuracy: 1.0000
Iteration: 17411, Training Loss: 0.0002, Training Accuracy: 1.0000
Iteration: 17421, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 17431, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 17441, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 17451, Training Loss: 0.0009, Training Accuracy: 1.0000
Iteration: 17461, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 17471, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 17481, Training Loss: 0.0003, Training Accuracy: 1.0000
Iteration: 17491, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 17501, Training Loss: 0.0003, Training Accuracy: 1.0000
Iteration: 17511, Training Loss: 0.0008, Training Accuracy: 1.0000
Iteration: 17521, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 17531, Training Loss: 0.0009, Training Accuracy: 1.

Iteration: 18621, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 18631, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 18641, Training Loss: 0.0012, Training Accuracy: 1.0000
Iteration: 18651, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 18661, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 18671, Training Loss: 0.0005, Training Accuracy: 1.0000
Iteration: 18681, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 18691, Training Loss: 0.0002, Training Accuracy: 1.0000
Iteration: 18701, Training Loss: 0.0102, Training Accuracy: 0.9922
Iteration: 18711, Training Loss: 0.0006, Training Accuracy: 1.0000
Iteration: 18721, Training Loss: 0.0002, Training Accuracy: 1.0000
Iteration: 18731, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 18741, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 18751, Training Loss: 0.0009, Training Accuracy: 1.0000
Iteration: 18761, Training Loss: 0.0001, Training Accuracy: 1.

Iteration: 19851, Training Loss: 0.0006, Training Accuracy: 1.0000
Iteration: 19861, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 19871, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 19881, Training Loss: 0.0008, Training Accuracy: 1.0000
Iteration: 19891, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 19901, Training Loss: 0.0005, Training Accuracy: 1.0000
Iteration: 19911, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 19921, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 19931, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 19941, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 19951, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 19961, Training Loss: 0.0009, Training Accuracy: 1.0000
Iteration: 19971, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 19981, Training Loss: 0.0039, Training Accuracy: 1.0000
Iteration: 19991, Training Loss: 0.0003, Training Accuracy: 1.

Iteration: 21081, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 21091, Training Loss: 0.0040, Training Accuracy: 1.0000
Iteration: 21101, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 21111, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 21121, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 21131, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 21141, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 21151, Training Loss: 0.0005, Training Accuracy: 1.0000
Iteration: 21161, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 21171, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 21181, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 21191, Training Loss: 0.0001, Training Accuracy: 1.0000
Iteration: 21201, Training Loss: 0.0000, Training Accuracy: 1.0000
Iteration: 21211, Training Loss: 0.0033, Training Accuracy: 1.0000
Iteration: 21221, Training Loss: 0.0001, Training Accuracy: 1.